In [1]:
import pandas            as pd
import numpy             as np
import statsmodels.api   as sm
import seaborn           as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

sns.set()

### Pre-DataAnalise

In [2]:
data_raw = pd.read_csv("KaggleV2-May-2016.csv")

In [3]:
data_raw.describe(include="all")

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
count,1.105270e+05,1.105270e+05,110527,110527,110527,110527.000000,110527,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527
unique,NaN,NaN,2,103549,27,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,F,2016-05-06T07:09:54Z,2016-06-06T00:00:00Z,NaN,JARDIM CAMBURI,NaN,NaN,NaN,NaN,NaN,NaN,No
freq,NaN,NaN,71840,24,4692,NaN,7717,NaN,NaN,NaN,NaN,NaN,NaN,88208
mean,1.474963e+14,5.675305e+06,NaN,NaN,NaN,37.088874,NaN,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026,NaN
std,2.560949e+14,7.129575e+04,NaN,NaN,NaN,23.110205,NaN,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873,NaN
min,3.921784e+04,5.030230e+06,NaN,NaN,NaN,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,4.172614e+12,5.640286e+06,NaN,NaN,NaN,18.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,3.173184e+13,5.680573e+06,NaN,NaN,NaN,37.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,9.439172e+13,5.725524e+06,NaN,NaN,NaN,55.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN


We have 37920 unique patiant, so I decided remouve duplcate patiants, notwithstanding they had diferents scheduleds id.

In [4]:
data_raw["AppointmentID"].nunique()

110527

In [5]:
data_raw = data_raw.copy()
data_raw.drop_duplicates(subset ="PatientId", 
                     keep = False, inplace = True)
data_raw["AppointmentID"].nunique()

37920

## Find days to Appointment

In [6]:
data_diferencedays = data_raw.copy()
data_diferencedays["ScheduledDayDATEFORMAT"] = (data_diferencedays["ScheduledDay"])
data_diferencedays["ScheduledDayDATEFORMAT"] = pd.to_datetime(data_diferencedays.ScheduledDayDATEFORMAT)
data_diferencedays["AppointmentDayDATEFORMAT"] = (data_diferencedays["AppointmentDay"])
data_diferencedays["AppointmentDayDATEFORMAT"] = pd.to_datetime(data_diferencedays.AppointmentDayDATEFORMAT)
data_diferencedays["DaysToAppointment"] = (data_diferencedays["AppointmentDayDATEFORMAT"] - data_diferencedays["ScheduledDayDATEFORMAT"]).dt.days
data_diferencedays.loc[data_diferencedays.DaysToAppointment < 0, 'DaysToAppointment'] = 0
data_diferencedays.describe(include="all")

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDayDATEFORMAT,AppointmentDayDATEFORMAT,DaysToAppointment
count,3.792000e+04,3.792000e+04,37920,37920,37920,37920.000000,37920,37920.000000,37920.000000,37920.000000,37920.000000,37920.000000,37920.000000,37920,37920,37920,37920.000000
unique,NaN,NaN,2,37232,27,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,2,37232,27,NaN
top,NaN,NaN,F,2016-03-29T10:43:41Z,2016-06-06T00:00:00Z,NaN,JARDIM CAMBURI,NaN,NaN,NaN,NaN,NaN,NaN,No,2016-03-29 10:43:41+00:00,2016-06-06 00:00:00+00:00,NaN
freq,NaN,NaN,23675,10,1677,NaN,2423,NaN,NaN,NaN,NaN,NaN,NaN,30802,10,1677,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-07 10:40:59+00:00,2016-04-29 00:00:00+00:00,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-08 20:07:23+00:00,2016-06-08 00:00:00+00:00,NaN
mean,1.475646e+14,5.668888e+06,NaN,NaN,NaN,36.784362,NaN,0.087236,0.197178,0.069409,0.022310,0.019146,0.335179,NaN,NaN,NaN,11.313555
std,2.552804e+14,7.749066e+04,NaN,NaN,NaN,23.831285,NaN,0.282185,0.397873,0.254152,0.147692,0.151480,0.472059,NaN,NaN,NaN,17.005456
min,3.921784e+04,5.134197e+06,NaN,NaN,NaN,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
25%,4.172537e+12,5.629586e+06,NaN,NaN,NaN,16.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000


### Clean outliers

In [7]:
q = 90 #data_raw['Age'].quantile(0.90)#90#
data_diferencedays = data_diferencedays[data_diferencedays["Age"]<q]
data_diferencedays = data_diferencedays[data_diferencedays["Age"]> 0]

q = 40 #data_diferencedays['DaysToAppointment'].quantile(0.99)
data_diferencedays = data_diferencedays[data_diferencedays["DaysToAppointment"]<q]
data_diferencedays.describe(include="all")

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDayDATEFORMAT,AppointmentDayDATEFORMAT,DaysToAppointment
count,3.430500e+04,3.430500e+04,34305,34305,34305,34305.000000,34305,34305.000000,34305.000000,34305.000000,34305.000000,34305.000000,34305.000000,34305,34305,34305,34305.000000
unique,NaN,NaN,2,33706,27,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,2,33706,27,NaN
top,NaN,NaN,F,2016-05-06T07:09:54Z,2016-06-06T00:00:00Z,NaN,MARIA ORTIZ,NaN,NaN,NaN,NaN,NaN,NaN,No,2016-05-06 07:09:54+00:00,2016-06-06 00:00:00+00:00,NaN
freq,NaN,NaN,21479,9,1516,NaN,1893,NaN,NaN,NaN,NaN,NaN,NaN,28138,9,1516,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-21 07:10:20+00:00,2016-04-29 00:00:00+00:00,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-08 20:07:23+00:00,2016-06-08 00:00:00+00:00,NaN
mean,1.476184e+14,5.681121e+06,NaN,NaN,NaN,37.281941,NaN,0.092290,0.201778,0.071972,0.023816,0.019560,0.314065,NaN,NaN,NaN,7.990118
std,2.551562e+14,6.071691e+04,NaN,NaN,NaN,22.993049,NaN,0.289439,0.401333,0.258445,0.152477,0.153272,0.464149,NaN,NaN,NaN,10.668220
min,3.921784e+04,5.491952e+06,NaN,NaN,NaN,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
25%,4.178387e+12,5.639981e+06,NaN,NaN,NaN,17.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000


## Whats day of week was the appointment:

In [8]:
data_weekname = data_diferencedays.copy()

In [9]:
data_weekname["WeekName"] = data_weekname["AppointmentDayDATEFORMAT"].dt.weekday_name
data_weekname["Month"] = data_weekname["AppointmentDayDATEFORMAT"].dt.month_name()

In [10]:
data_weekname.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDayDATEFORMAT,AppointmentDayDATEFORMAT,DaysToAppointment,WeekName,Month
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,0,Friday,April
6,7.336882e+14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,Yes,2016-04-27 15:05:12+00:00,2016-04-29 00:00:00+00:00,1,Friday,April
8,5.639473e+13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,No,2016-04-29 08:02:16+00:00,2016-04-29 00:00:00+00:00,0,Friday,April
9,7.812456e+13,5629123,F,2016-04-27T12:48:25Z,2016-04-29T00:00:00Z,19,CONQUISTA,0,0,0,0,0,0,No,2016-04-27 12:48:25+00:00,2016-04-29 00:00:00+00:00,1,Friday,April
10,7.345362e+14,5630213,F,2016-04-27T14:58:11Z,2016-04-29T00:00:00Z,30,NOVA PALESTINA,0,0,0,0,0,0,No,2016-04-27 14:58:11+00:00,2016-04-29 00:00:00+00:00,1,Friday,April


## Create binary to GENDER and No-Show 

In [11]:
data_toBinary = data_weekname.copy()
data_toBinary["No-show"] = data_toBinary["No-show"].map({"No":0, "Yes":1})
data_toBinary["Gender"] = data_toBinary["Gender"].map({"F":1, "M":0})
data_toBinary.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDayDATEFORMAT,AppointmentDayDATEFORMAT,DaysToAppointment,WeekName,Month
4,8.841186e+12,5642494,1,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,0,Friday,April
6,7.336882e+14,5630279,1,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27 15:05:12+00:00,2016-04-29 00:00:00+00:00,1,Friday,April
8,5.639473e+13,5638447,1,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29 08:02:16+00:00,2016-04-29 00:00:00+00:00,0,Friday,April
9,7.812456e+13,5629123,1,2016-04-27T12:48:25Z,2016-04-29T00:00:00Z,19,CONQUISTA,0,0,0,0,0,0,0,2016-04-27 12:48:25+00:00,2016-04-29 00:00:00+00:00,1,Friday,April
10,7.345362e+14,5630213,1,2016-04-27T14:58:11Z,2016-04-29T00:00:00Z,30,NOVA PALESTINA,0,0,0,0,0,0,0,2016-04-27 14:58:11+00:00,2016-04-29 00:00:00+00:00,1,Friday,April


## Creating Dummies to Neighbourhood, weekname and month

In [12]:
data_w_dummies = pd.get_dummies(data_toBinary, columns=["Neighbourhood","WeekName","Month"],drop_first=True)
data_w_dummies.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Scholarship,Hipertension,Diabetes,Alcoholism,...,Neighbourhood_TABUAZEIRO,Neighbourhood_UNIVERSITÁRIO,Neighbourhood_VILA RUBIM,WeekName_Monday,WeekName_Saturday,WeekName_Thursday,WeekName_Tuesday,WeekName_Wednesday,Month_June,Month_May
4,8.841186e+12,5642494,1,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,7.336882e+14,5630279,1,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5.639473e+13,5638447,1,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,7.812456e+13,5629123,1,2016-04-27T12:48:25Z,2016-04-29T00:00:00Z,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,7.345362e+14,5630213,1,2016-04-27T14:58:11Z,2016-04-29T00:00:00Z,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#data_w_dummies.columns.values

### Finding TARGET and Inputs data

In [14]:
data2 = data_w_dummies.copy()
data2

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Scholarship,Hipertension,Diabetes,Alcoholism,...,Neighbourhood_TABUAZEIRO,Neighbourhood_UNIVERSITÁRIO,Neighbourhood_VILA RUBIM,WeekName_Monday,WeekName_Saturday,WeekName_Thursday,WeekName_Tuesday,WeekName_Wednesday,Month_June,Month_May
4,8.841186e+12,5642494,1,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,7.336882e+14,5630279,1,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5.639473e+13,5638447,1,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,7.812456e+13,5629123,1,2016-04-27T12:48:25Z,2016-04-29T00:00:00Z,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,7.345362e+14,5630213,1,2016-04-27T14:58:11Z,2016-04-29T00:00:00Z,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110501,4.544252e+11,5758133,0,2016-06-01T10:19:12Z,2016-06-01T00:00:00Z,44,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
110511,8.235996e+11,5786742,1,2016-06-08T08:50:20Z,2016-06-08T00:00:00Z,14,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
110512,9.876246e+13,5786368,1,2016-06-08T08:20:01Z,2016-06-08T00:00:00Z,41,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
110513,8.674778e+13,5785964,0,2016-06-08T07:52:55Z,2016-06-08T00:00:00Z,2,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [15]:
target = data2[["No-show"]]
inputs = data2.drop(['No-show','PatientId', 'AppointmentID', 'ScheduledDay'
                     ,'AppointmentDay','ScheduledDayDATEFORMAT', 'AppointmentDayDATEFORMAT',
                     'Month_May','Month_June'], axis=1)
#inputs = data2[['DaysToAppointment',"Age","Gender"]]
#inputs

In [16]:
'''from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)
inputs_scaled = scaler.transform(inputs)'''

'from sklearn.preprocessing import StandardScaler\n\nscaler = StandardScaler()\nscaler.fit(inputs)\ninputs_scaled = scaler.transform(inputs)'

### Split data in train and test

In [17]:
x_train, x_test, y_train, y_test = train_test_split(inputs, target, test_size=0.3, random_state=42)
y_train

,No-show
41456,0
85848,1
67234,0
73473,0
26939,0
...,...
56041,0
23311,1
39393,0
2657,1


### Plot of Age vs No-Show

plt.scatter(x_train['Age'],y_train,color='C0',  alpha=0.2)
plt.xlabel("Age", fontsize=20)
plt.ylabel('No-show', fontsize=20)
plt.show()

In [18]:
inputs_train = sm.add_constant(x_train)#.to_numpy())
reg_log = sm.Logit(y_train,inputs_train)
results_log = reg_log.fit(maxiter=1400, method='cg')
results_log.summary()


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


         Current function value: 0.437197
         Iterations: 1400
         Function evaluations: 3173
         Gradient evaluations: 3170


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                No-show   No. Observations:                24013
Model:                          Logit   Df Residuals:                    23918
Method:                           MLE   Df Model:                           94
Date:                Fri, 29 Nov 2019   Pseudo R-squ.:                 0.07337
Time:                        11:56:00   Log-Likelihood:                -10498.
converged:                      False   LL-Null:                       -11330.
Covariance Type:            nonrobust   LLR p-value:                3.796e-285
=============================================================================================================
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                        -1.5736      1.573     -1.000      0.317      -4.657       1.510
Gender                                        0.0168      0.037      0.456      0.648      -0.055       0.089
Age                                          -0.0075      0.001     -7.939      0.000      -0.009      -0.006
Scholarship                                   0.1042      0.059      1.755      0.079      -0.012       0.221
Hipertension                                 -0.1565      0.059     -2.663      0.008      -0.272      -0.041
Diabetes                                      0.1093      0.080      1.372      0.170      -0.047       0.265
Alcoholism                                    0.3034      0.108      2.797      0.005       0.091       0.516
Handcap                                       0.0098      0.123      0.080      0.937      -0.231       0.250
SMS_received                                  0.2909      0.040      7.295      0.000       0.213       0.369
DaysToAppointment                             0.0495      0.002     28.195      0.000       0.046       0.053
Neighbourhood_ANDORINHAS                     -0.1601      1.579     -0.101      0.919      -3.255       2.935
Neighbourhood_ANTÔNIO HONÓRIO                -0.8595      1.605     -0.535      0.592      -4.006       2.287
Neighbourhood_ARIOVALDO FAVALESSA             0.0528      1.602      0.033      0.974      -3.087       3.193
Neighbourhood_BARRO VERMELHO                  0.4076      1.598      0.255      0.799      -2.725       3.541
Neighbourhood_BELA VISTA                      0.0261      1.578      0.017      0.987      -3.066       3.118
Neighbourhood_BENTO FERREIRA                  0.0171      1.583      0.011      0.991      -3.085       3.119
Neighbourhood_BOA VISTA                       0.0544      1.603      0.034      0.973      -3.087       3.195
Neighbourhood_BONFIM                         -0.0233      1.576     -0.015      0.988      -3.112       3.066
Neighbourhood_CARATOÍRA                       0.3553      1.576      0.225      0.822      -2.733       3.444
Neighbourhood_CENTRO                         -0.0336      1.576     -0.021      0.983      -3.123       3.055
Neighbourhood_COMDUSA                         0.1687      1.613      0.105      0.917      -2.992       3.330
Neighbourhood_CONQUISTA                       0.1296      1.583      0.082      0.935      -2.974       3.233
Neighbourhood_CONSOLAÇÃO                     -0.2779      1.581     -0.176      0.860      -3.376       2.821
Neighbourhood_CRUZAMENTO                     -0.2227      1.583     -0.141      0.888      -3.326       2.880
Neighbourhood_DA PENHA                        0.0100      1.577      0.006      0.995      -3.081       3.101
Neighbourhood_DE LOURDES                     -0.4204      1.609     -0.261      0.794      -3.574       2.733
Neighbourhood_DO CABRAL                      -0.681

In [19]:
np.set_printoptions(formatter={"float": lambda x:"{0:0.2f}".format(x)})
results_log.predict()

array([0.11, 0.09, 0.09, ..., 0.11, 0.15, 0.12])

In [20]:
np.array(data2["No-show"])

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

### Table of results - Confunsion Matrix

In [21]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns = ["Predicted 0", "Predicted 1"]
cm_df = cm_df.rename(index={0:"Actual 0", 1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,19525.0,159.0
Actual 1,4223.0,106.0


### Results of Confusion Matrix --- Model's accuracy

In [22]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum()
accuracy_train

0.8175155124307667

In [23]:
x_train#['No-show']
#len(x_train.columns.values)

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,DaysToAppointment,Neighbourhood_ANDORINHAS,...,Neighbourhood_SÃO JOSÉ,Neighbourhood_SÃO PEDRO,Neighbourhood_TABUAZEIRO,Neighbourhood_UNIVERSITÁRIO,Neighbourhood_VILA RUBIM,WeekName_Monday,WeekName_Saturday,WeekName_Thursday,WeekName_Tuesday,WeekName_Wednesday
41456,0,51,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,1,0
85848,0,51,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
67234,1,63,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
73473,1,42,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
26939,0,66,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56041,1,35,0,0,0,1,0,1,4,0,...,1,0,0,0,0,0,0,0,0,1
23311,0,27,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
39393,1,49,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,1,0
2657,1,36,0,1,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
x_test_c = sm.add_constant(x_test)#.to_numpy())
y_test_c = y_test['No-show']

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [25]:
def confusion_matrix(data,actual_values,model):
        #Predict the values using the Logit model
        pred_values = model.predict(data)
        # Specify the bins 
        bins=np.array([0,0.5,1])
        # Create a histogram, where if values are between 0 and 0.5 tell will be considered 0
        # if they are between 0.5 and 1, they will be considered 1
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        # Calculate the accuracy
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        # Return the confusion matrix and the accuracy
        return cm, accuracy

In [26]:
confusion_matrix(x_test_c , y_test_c, results_log)


(array([[8367.00, 87.00],
        [1787.00, 51.00]]), 0.8179168286047416)

We had accuracy 81,79%